In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Conv2D,Flatten,MaxPooling2D,MaxPool2D,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Importing Files**

In [ ]:
train=pd.read_csv('../input/Kannada-MNIST/train.csv')
test=pd.read_csv('../input/Kannada-MNIST/test.csv')
sample_sub=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test=test.drop('id',axis=1)
test.head()

In [ ]:
y=train.label.value_counts()
y

In [ ]:
X_train=train.drop('label',axis=1)
Y_train=train.label

In [ ]:
K = len(set(Y_train))
X_train=X_train/255
test=test/255

In [ ]:
X_train=X_train.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)

In [ ]:
print(X_train.shape)
print(test.shape)

In [ ]:
#TO a numpy array
Y_train=to_categorical(Y_train)
Y_train

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_train,Y_train,random_state=42,test_size=0.2)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization(momentum=.15))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization(momentum=0.15))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization(momentum=.15))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr = 0.001),
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
model_1 = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,
                    callbacks=[learning_rate_reduction])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model_1.history['loss'],label='Loss')
plt.plot(model_1.history['val_loss'],label='Val_Loss')
plt.legend()

In [ ]:
plt.plot(model_1.history['accuracy'],label='Accuracy')
plt.plot(model_1.history['val_accuracy'],label='Val_Accuracy')
plt.legend()

In [ ]:
y_pre_test=model.predict(X_test)
y_pre_test=np.argmax(y_pre_test,axis=1)
y_test=np.argmax(y_test,axis=1)

**Plotting Confusion Matrix**

In [ ]:
conf=confusion_matrix(y_test,y_pre_test)
conf=pd.DataFrame(conf,index=range(0,10),columns=range(0,10))
conf

In [ ]:
#Plotting confusion Matrix
plt.figure(figsize=(12, 8))
ax = sns.heatmap(conf,annot=True, fmt="d",cmap="YlGnBu")

**Evaluating**

In [ ]:
test=pd.read_csv('../input/Kannada-MNIST/test.csv')

In [ ]:
test_id=test.id

test=test.drop('id',axis=1)
test=test/255
test=test.values.reshape(-1,28,28,1)
test.shape

In [ ]:
pred=model.predict(test)  
pred=np.argmax(pred,axis=1)

In [ ]:
sample_sub #sample_submission File

In [ ]:
sample_sub['label']=pred
sample_sub.to_csv('submission.csv',index=False)

In [ ]:
sample_sub